In [33]:
import pandas as pd
train = pd.read_csv("training.csv",header=0)
test = pd.read_csv("test.csv",header=0)
valid = pd.read_csv("validation.csv",header=0)

#type(train['text'])

pandas.core.series.Series

In [22]:
#tokens
from nltk.tokenize import word_tokenize
def token(content):
    return word_tokenize(content,"english")

#remove stop words
from nltk.corpus import stopwords
final_words=[]
def rm_stopwords(words):
   final_words = [word for word in words if word not in stopwords.words("english")]
   final_words_string = ''.join(final_words)
   return final_words_string

# stemming + lemmatization
from nltk.stem import SnowballStemmer 
snowball = SnowballStemmer(language='english')

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

def many_stem_lemma(list_of_words):
    new_list =''
    for word in list_of_words:
      stem =  snowball.stem(word)
      lemma = lemmatizer.lemmatize(stem,pos='v')
      new_list += lemma
    return new_list

#combining all
import re
def transf(x):
    lower_content = x.lower()
    punct_content = re.sub(r'[^\w\s]', '', lower_content)

    L = many_stem_lemma(rm_stopwords(token(punct_content)))
    return L

In [23]:

train['text'] = train['text'].apply(lambda x: transf(x))
test['text'] = test['text'].apply(lambda x: transf(x))
valid['text'] = valid['text'].apply(lambda x: transf(x))



In [24]:
#train.tail()

In [25]:
x_train = train['text']
y_train = train['label']
x_test = test['text']
y_test = test['label']
x_valid = valid['text']
y_valid = valid['label']

#x_train

In [26]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

total_reviews=[]
total_reviews.append(x_train.tolist())
total_reviews.append(x_test.tolist())
total_reviews.append(x_valid.tolist())

tokenizer_obj = Tokenizer()
tokenizer_obj.fit_on_texts(total_reviews)

max_length = max([len(s) for s in total_reviews])
vocab_size = len(tokenizer_obj.word_index) + 1

x_train_tokens = tokenizer_obj.texts_to_sequences(x_train)
x_test_tokens = tokenizer_obj.texts_to_sequences(x_test)
x_valid_tokens = tokenizer_obj.texts_to_sequences(x_valid)

x_train_pad = pad_sequences(x_train_tokens, maxlen=max_length, padding='post')
x_test_pad = pad_sequences(x_test_tokens, maxlen=max_length, padding='post')
x_valid_pad = pad_sequences(x_valid_tokens, maxlen=max_length, padding='post')


In [7]:
#len(x_train_pad)

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# First Model 
class RNN_model1(nn.Module):
    def __init__(self):
        super(RNN_model1, self).__init__()
        self.lstm1 =nn.LSTM(16,32,batch_first=True)
        self.lstm2 =nn.LSTM(32,64,batch_first=True)
        self.lstm3 =nn.LSTM(64,128,batch_first=True)
        self.flat =nn.Flatten()
        self.dense =nn.Linear(128, 6)
        
    def forward(self,x):
        outp1, _ = self.lstm1(x)
        outp2, _ = self.lstm2(outp1)
        outp3, _ = self.lstm3(outp2)
        outp = self.flat(outp3.squeeze())
        outp = self.dense(outp)
        probs = F.softmax(outp, dim=1)  
        predicted_class = torch.argmax(probs, dim=1)
        return predicted_class.to(torch.float32)


In [10]:
model = RNN_model1()


In [11]:
loss_funct = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

In [12]:
x_train_tensor = torch.tensor(x_train_pad, dtype=torch.float32)
#x_train_tensor = x_train_tensor.unsqueeze(0)
#x_train_tensor = x_train_tensor.view(-1,16000)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32, requires_grad=True)


In [13]:
#len(x_train_tensor)

In [61]:
from torch.utils.data import TensorDataset, DataLoader
train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
train_dataloader = DataLoader(train_dataset, batch_size=10, shuffle=True)

In [62]:
'''for x, y in train_dataloader:
    print(y.shape)'''

'for x, y in train_dataloader:\n    print(y.shape)'

In [63]:
from torchmetrics.functional import accuracy

epochs= 10
for epoch in range(epochs):
    # intitialisations
    total_loss = 0
    total_acc = 0
    model.train()
    ln = 0
    for ids, (x,y) in enumerate(train_dataloader):
        pred = model(x)
        loss = loss_funct(pred,y)
        acc = accuracy(pred, y, task="multiclass", num_classes=6)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_acc += acc.item()
        ln +=1
        
    avg_loss = total_loss / ln 
    avg_acc = total_acc / ln 
    print("epoch: ",epoch+1,"  loss: ",avg_loss,"  accuracy: ",avg_acc)


epoch:  1   loss:  36.05704524517059   accuracy:  0.29162500604521485
epoch:  2   loss:  36.05704521894455   accuracy:  0.29162500612903386
epoch:  3   loss:  36.05704517453909   accuracy:  0.2916250058123842
epoch:  4   loss:  36.05704520523548   accuracy:  0.2916250059939921
epoch:  5   loss:  36.057045257091524   accuracy:  0.29162500591482965
epoch:  6   loss:  36.057045242488385   accuracy:  0.2916250058542937
epoch:  7   loss:  36.0570452389121   accuracy:  0.2916250059893355
epoch:  8   loss:  36.05704519212246   accuracy:  0.2916250059613958
epoch:  9   loss:  36.057045176029206   accuracy:  0.2916250059893355
epoch:  10   loss:  36.057045287787915   accuracy:  0.2916250057751313


In [64]:

# Second Model 
class RNN_model2(nn.Module):
    def __init__(self):
        super(RNN_model2, self).__init__()
        self.gru1 =nn.GRU(16000,32,batch_first=True)
        self.gru2 =nn.GRU(32,64,batch_first=True)
        self.gru3 =nn.GRU(64,128,batch_first=True)
        self.flat =nn.Flatten()
        self.dense =nn.Linear(128, 6)
        
    def forward(self,x):
        outp1, _ = self.gru1(x)
        outp2, _ = self.gru2(outp1)
        outp3, _ = self.gru3(outp2)
        outp = self.flat(outp3.squeeze())
        outp = self.dense(outp)
        probs = F.softmax(outp, dim=1)  
        predicted_class = torch.argmax(probs, dim=1)
        return predicted_class.to(torch.float32)
        

In [65]:
model2 = RNN_model2()
optimizer = torch.optim.SGD(model2.parameters(), lr=0.001)

In [66]:
epochs= 10
for epoch in range(epochs):
    # intitialisations
    total_loss = 0
    total_acc = 0
    model2.train()
    ln = 0
    for ids, (x,y) in enumerate(train_dataloader):
        pred = model2(x)
        loss = loss_funct(pred,y)
        acc = accuracy(pred, y, task="multiclass", num_classes=6)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_acc += acc.item()
        ln +=1
        
    avg_loss = total_loss / ln 
    avg_acc = total_acc / ln 
    print("epoch: ",epoch+1,"  loss: ",avg_loss,"  accuracy: ",avg_acc)


epoch:  1   loss:  36.057045247852805   accuracy:  0.035750000583939255
epoch:  2   loss:  36.057045160532   accuracy:  0.03575000056065619
epoch:  3   loss:  36.05704523533583   accuracy:  0.03575000055134296
epoch:  4   loss:  36.05704524487257   accuracy:  0.035750000583939255
epoch:  5   loss:  36.05704522818327   accuracy:  0.03575000057462603
epoch:  6   loss:  36.05704524934292   accuracy:  0.0357500005653128
epoch:  7   loss:  36.05704527974129   accuracy:  0.03575000057928264
epoch:  8   loss:  36.05704522043467   accuracy:  0.0357500005653128
epoch:  9   loss:  36.05704526782036   accuracy:  0.03575000055599958
epoch:  10   loss:  36.057045251727104   accuracy:  0.03575000056065619


In [ ]:
#y_train

In [15]:
#y_train.head()

0    0
1    0
2    3
3    2
4    3
Name: label, dtype: int64